In [1]:
pip install transformers torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import os

api_key = os.getenv('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set. Please set it and try again.")

# Function to generate interview question and answer
def generate_question_answer(prompt, max_tokens=300, temperature=0.7):
    response = openai.chat.completions.create(
       model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature
    )
    return response['choices'][0]['message']['content'].strip()

# Generate multiple interview questions and answers
question_answer_pairs = []
question_prompt = "Generate an interview question and a detailed answer for a candidate applying for a job in the field of [FIELD]."

fields = ["project management", "software engineering", "marketing", "data analysis", "customer service", "finance", "education", "graphic design", "human resources", "operations management"]

for field in fields:
    for _ in range(40):  # 40 questions and answers per field to generate a total of 400
        prompt = question_prompt.replace("[FIELD]", field)
        response = generate_question_answer(prompt)
        question_answer_pairs.append(response)

# Save questions and answers to a text file
with open('generated_interview_qas.txt', 'w') as file:
    for pair in question_answer_pairs:
        file.write(pair + '\n\n')

print("Generated 400 interview questions and answers and saved to 'generated_interview_qas.txt'.")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

I wanted to Generate responses via Open AI API as I tried in the code above, but I am getting Error code:429 as I am unable to pay for it. Therefore I have resorted to using chatGPT directly to generate responses(Interview Questions and Answers) and make a .csv file for the same.
p.s. I have used GPT-2 as well as my model for above code but couldn't generate accurate responses as needed cause it contained too many grammatical anamolies.

In [8]:
import os
import tkinter as tk
from tkinter import messagebox
from textblob import TextBlob
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Load English language model for spaCy
nlp = spacy.load('en_core_web_sm')


In [9]:
def analyze_sentiment(response):
    blob = TextBlob(response)
    sentiment_score = blob.sentiment.polarity
    
    if sentiment_score > 0:
        sentiment = "positive"
    elif sentiment_score < 0:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    
    return sentiment, sentiment_score

def extract_key_phrases(response):
    doc = nlp(response)
    key_phrases = []
    
    for chunk in doc.noun_chunks:
        phrase = ' '.join(token.text for token in chunk if not token.is_stop)
        key_phrases.append(phrase)
    
    return key_phrases


In [13]:
def submit_question_answer():
    question = entry_question.get().strip()  # Ensure entry exists and is accessible
    answer = entry_answer.get().strip()      # Ensure entry exists and is accessible
    
    if question and answer:
        # Perform sentiment analysis and key phrase extraction
        sentiment, sentiment_score = analyze_sentiment(answer)
        key_phrases = extract_key_phrases(answer)
        
        # Display the results
        result_text = f"Sentiment: {sentiment}\n\nKey Phrases: {key_phrases}\n\n"
        result_text += f"Overall Quality Score: {sentiment_score + len(key_phrases) * 0.1:.2f}"
        
        messagebox.showinfo("Analysis Results", result_text)
        
        # Optionally, save the new question and answer to a file or database
        
        # Clear the input fields
        entry_question.delete(0, tk.END)
        entry_answer.delete(0, tk.END)
    else:
        messagebox.showerror("Error", "Please enter both question and answer.")

In [18]:
# Create the main window
root = tk.Tk()
root.title("Interview Question and Answer Input")

# Question input
label_question = tk.Label(root, text="Enter Question:")
label_question.pack(pady=10)
entry_question = tk.Entry(root, width=50)
entry_question.pack()

# Answer input
label_answer = tk.Label(root, text="Enter Answer:")
label_answer.pack(pady=10)
entry_answer = tk.Entry(root, width=50)
entry_answer.pack()

# Submit button
submit_button = tk.Button(root, text="Submit", command=submit_question_answer)
submit_button.pack(pady=20)

# Run the main loop
root.mainloop()

Above 3 Cells of code is for Manually adding your Response(Question and Answer) in an Interactive window to return corresponding Sentiment, Key Phrases and Quality Score as well while below code is for sentiment analysis of already present responses in responses.txt file in same folder. Run every cell post importing necessary libraries for the interactive window to pop up.

In [15]:
def process_responses_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        responses = file.read().strip().split('\n\n')
    
    results = []
    
    for response_block in responses:
        question, answer = response_block.split('\n')
        
        # Analyze sentiment
        sentiment, sentiment_score = analyze_sentiment(answer)
        
        # Extract key phrases
        key_phrases = extract_key_phrases(answer)
        
        # Overall quality assessment
        quality_score = sentiment_score + len(key_phrases) * 0.1  # Example combination
        
        results.append({
            'question': question,
            'answer': answer,
            'sentiment': sentiment,
            'key_phrases': key_phrases,
            'quality_score': quality_score
        })
    
    return results

In [16]:
def main():
    # Load existing responses from file
    existing_responses = process_responses_from_file('responses.txt')
    return existing_responses

In [17]:
if __name__ == "__main__":
    results = main()
    
    # Display results or save them as needed
    for idx, result in enumerate(results, start=1):
        print(f"Question {idx}: {result['question']}")
        print(f"Answer: {result['answer']}")
        print(f"Sentiment: {result['sentiment']}")
        print(f"Key Phrases: {result['key_phrases']}")
        print(f"Quality Score: {result['quality_score']:.2f}\n")

Question 1: Question: How do you handle tight deadlines?
Answer: Answer: I prioritize tasks and manage my time efficiently. However, sometimes the pressure causes me to make mistakes, which I then have to correct, impacting the overall timeline.
Sentiment: neutral
Key Phrases: ['', 'tasks', 'time', 'pressure', '', 'mistakes', '', '', 'overall timeline']
Quality Score: 0.90

Question 2: Question: Can you describe a time when you faced a major challenge at work?
Answer: Answer: I faced a significant challenge when a project I was leading fell behind schedule due to unforeseen technical issues. Despite my efforts, we were unable to meet the original deadline, which disappointed our client.
Sentiment: negative
Key Phrases: ['', 'significant challenge', 'project', '', 'schedule', 'unforeseen technical issues', 'efforts', '', 'original deadline', '', 'client']
Quality Score: 0.95

Question 3: Question: How do you approach learning new skills?
Answer: Answer: I am proactive in learning new sk

Thank You!